<a href="https://colab.research.google.com/github/daotismart/Colaboratory/blob/master/my_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!nvidia-smi

Tue Nov  5 15:42:45 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    56W / 149W |     69MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/device:GPU:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Ensure that TF can detect the GPU.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

SyntaxError: ignored

In [1]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.100.79.42:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 9192024358033324666),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11030546621385318037),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1103778993579264395),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10678666028221508971),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2111134298377998511),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4830106975220869580),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 17937469497975113670),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 2133091406477965277),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 5676475190335

In [2]:
N = 4096
COUNT = 100
import time

def flops():
  x = tf.random_uniform([N, N])
  y = tf.random_uniform([N, N])
  def _matmul(x, y):
    return tf.tensordot(x, y, axes=[[1], [0]]), y

  return tf.reduce_sum(
    tf.contrib.tpu.repeat(COUNT, _matmul, [x, y])
  )
  
tpu_ops = tf.contrib.tpu.batch_parallel(flops, [], num_shards=8)
  
session = tf.Session(tpu_address)
try:
  print('Warming up...')
  session.run(tf.contrib.tpu.initialize_system())
  session.run(tpu_ops)
  print('Profiling')
  start = time.time()
  session.run(tpu_ops)
  end = time.time()
  elapsed = end - start
  print(elapsed, 'TFlops: {:.2f}'.format(1e-12 * 8 * COUNT * 2*N*N*N / elapsed))
finally:
  session.run(tf.contrib.tpu.shutdown_system())
  session.close()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Warming up...
Profiling
0.6749606132507324 TFlops: 162.90


In [0]:
from google.colab import drive
drive.mount('/content/drive')

file_name	size	link
kfbreader.zip	59.8MB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/kfbreader.zip
labels.zip	210.29KB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/labels.zip
neg_0.zip	10.74GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip
neg_1.zip	10.24GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_1.zip
neg_2.zip	11.55GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_2.zip
neg_3.zip	11.11GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_3.zip
neg_4.zip	11.25GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_4.zip
neg_5.zip	11.57GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_5.zip
pos_0.zip	13.01GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_0.zip
pos_1.zip	12.68GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_1.zip
pos_2.zip	12.85GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_2.zip
pos_3.zip	12.74GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_3.zip
pos_4.zip	12.81GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_4.zip
pos_5.zip	12.84GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_5.zip
pos_6.zip	13.36GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_6.zip
pos_7.zip	13.41GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_7.zip
pos_8.zip	12.97GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_8.zip
pos_9.zip	14.48GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/pos_9.zip
test_0.zip	12.03GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/test_0.zip
test_1.zip	12.5GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/test_1.zip
test_2.zip	12.64GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/test_2.zip
test_3.zip	11.4GB	http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/test_3.zip


In [12]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/kfbreader.zip cancer_data

--2019-11-05 15:59:53--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/kfbreader.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77483037 (74M) [application/zip]
Saving to: ‘kfbreader.zip’

kfbreader.zip       100%[===================>]  73.89M  1.32MB/s    in 55s     

2019-11-05 16:00:50 (1.34 MB/s) - ‘kfbreader.zip’ saved [77483037/77483037]

--2019-11-05 16:00:50--  http://cancer_data/
Resolving cancer_data (cancer_data)... failed: Name or service not known.
wget: unable to resolve host address ‘cancer_data’
FINISHED --2019-11-05 16:00:50--
Total wall clock time: 57s
Downloaded: 1 files, 74M in 55s (1.34 MB/s)


In [13]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/labels.zip neg_0.zip cancer_data/ &

--2019-11-05 16:01:22--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/labels.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215337 (210K) [application/zip]
Saving to: ‘labels.zip’

labels.zip          100%[===================>] 210.29K   254KB/s    in 0.8s    

2019-11-05 16:01:23 (254 KB/s) - ‘labels.zip’ saved [215337/215337]

--2019-11-05 16:01:23--  http://neg_0.zip/
Resolving neg_0.zip (neg_0.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘neg_0.zip’
--2019-11-05 16:01:23--  http://cancer_data/
Resolving cancer_data (cancer_data)... failed: Name or service not known.
wget: unable to resolve host address ‘cancer_data’
FINISHED --2019-11-05 16:01:23--

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip neg_1.zip ./cancer_data/ &

--2019-11-05 16:02:49--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11531655505 (11G) [application/zip]
Saving to: ‘neg_0.zip’

neg_0.zip             3%[                    ] 348.72M  1.19MB/s    eta 2h 26m 

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip neg_2.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip neg_3.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip neg_4.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip neg_5.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip pos_0.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip pos_1.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip pos_2.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip pos_3.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip pos_4.zip ./cancer_data/ &

In [0]:
!wget  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231757/neg_0.zip pos_5.zip ./cancer_data/ &